In [18]:
import pandas as pd
import numpy as np

# GUI - Under Construction

This creates the dialogue box where the user enters the project information

Needs to be able to import the customer load file from csv
Inputs:
Cost of solar (dollar per w), Cost of storage (dollar per kWh)

In [12]:
import tkinter
from tkinter import *
fields = 'Storage Size kW','Storage Hours', 'Storage CapEx', 'Years of Use', 'Tenor', 'Interest Rate', 'PV Size kW', 'PV Cost per Watt','PV Irradiation kWh per kW','PV Years', 'Diesel Size kW', 'Diesel CapEx','Diesel Fuel Cost','Diesel Price','Diesel Years','Diesel Max Throughput','Electricity Price'

def fetch(entries):
    for entry in entries:
        field = entry[0]
        text=entry[1].get()
        print('%s:"%s"'%(field,text))
        
def makeform(root,fields):
    entries=[]
    for field in fields:
        row=Frame(root)
        lab=Label(row,width=30,text=field,anchor='w')
        ent=Entry(row)
        row.pack(side=TOP,fill=X,padx=5,pady=5)
        lab.pack(side=LEFT)
        ent.pack(side=RIGHT,expand=YES,fill=X)
        entries.append((field,ent))
    return entries
    
if __name__ == '__main__':
    root=Tk()
    ents=makeform(root,fields)
    root.bind('<Return>'),(lambda event,e=ents:fetch(e))
    b1=Button(root,text='Show',command=(lambda e=ents:fetch(e)))
    b1.pack(side=LEFT,padx=5,pady=5)
    b2=Button(root,text='Quit',command=root.destroy)
    b2.pack(side=LEFT,padx=5,pady=5)
#    b3=Button(root,text='Submit',command=)
    root.mainloop()

# Import Datasets: PV Output and Customer Load

Note: Need to find data that better suits this scenario
Note2: Need to automate the PV data pull here
Note3: Make customer load read from excel

In [ ]:
#PV Output
PVOutput = pd.read_excel('/Users/ajames2/Desktop/Python/PVOutput.xlsx')

#Customer Load
CustomerLoad = pd.read_excel('/Users/ajames2/Desktop/Python/CustomerLoad.xlsx')

# Scenario Inputs: Diesel, Storage, PV, and Electricity Rates

Note: Need to get all this into Tkinter
Note2: Need to add in dynamic approach for lifetime on diesel and storage based on throughput
Note3: Need to add in roundtrip efficiency
Note4: Add in solar costs and multi-year

In [ ]:
#Input Data

#Storage
storage_size_kW = float(250)
storage_hours = float(4)
storage_capex = float(250000)
years_of_use = float(10)
storage_max_throughput = float(2700*storage_size_kW*storage_hours)

#Storage Financing
tenor = 10
interest_rate = 0.20

#Storage Annual Payment
storage_annual_payment = (storage_capex *(interest_rate * (1 + interest_rate) ** tenor) /
                         ((1 + interest_rate) ** tenor - 1))

#solar LCOE
pv_size_kW = 150
pv_cpw = 2.5 #pv cost per watt
pv_capex = (pv_size_kW * pv_cpw * 1000)
pv_irradiation = 1400
pv_years = 20
pv_total_production = pv_irradiation * pv_years * pv_size_kW

#Solar LCOE
solar_lcoe = pv_capex / pv_total_production

#Diesel Generator
diesel_size_kW = float(250)
diesel_capex = float(100000)
diesel_fuel_cost =float(3.5)
diesel_price = float(.40)
diesel_years = 10
diesel_max_throughput = float(36000) #hours
    
#Electricity Price - TOU
        #00:01 - 11:00
        #11:01 - 15:00
        #15:01 - 24:00
    
#Electricity Price - Fixed
electricity_price = float(0.13)

# Clean Datasets and Create New Dataframe

In [ ]:
#Rename kWh columns
CustomerLoad.rename(columns={'Building 74 - kWh Total Electricity (kWh)':'Customer_Load'}, inplace=True)

#Convert to hourly
CustomerLoad['timestamp_hour'] = CustomerLoad['Timestamp'].apply(
    lambda x: pd.Timestamp(year=x.year,month=x.month,day=x.day,hour=x.hour))
CustomerLoad.pivot_table(index=['timestamp_hour'], 
                         values='Customer_Load', aggfunc='sum').reset_index()
del CustomerLoad['Timestamp']
EnergyMaster = CustomerLoad

# Create Required Columns for New Dataframe

In [ ]:
#Add electricity price
EnergyMaster['electricity_price'] = electricity_price

#create column for pv production
EnergyMaster['PV_Output'] = PVOutput

#create column to show customers load after they have consumed PV
EnergyMaster['PV_Output_Post_Load'] = (EnergyMaster['PV_Output']-EnergyMaster['Customer_Load']).clip_lower(0)

#create column for amount of PV left after customer load
EnergyMaster['Customer_Load_Post_PV'] = (EnergyMaster['Customer_Load']-EnergyMaster['PV_Output']).clip_lower(0)

#create storage state of charge in three parts

    #charge storage first from the remaining PV available
EnergyMaster['starting_storage_SOC_kWh'] = EnergyMaster['PV_Output_Post_Load'].cumsum().clip(0,storage_size_kW * storage_hours)
    
    #identify the potential charge from the grid capped by the storage units size
EnergyMaster['potential_grid_charge'] = ((storage_size_kW * storage_hours) - EnergyMaster['starting_storage_SOC_kWh']).cumsum().clip(0,storage_size_kW)
    
    #charge storage from the grid capped at the storage unit size
EnergyMaster['finished_storage_SOC_kWh'] = (EnergyMaster['starting_storage_SOC_kWh'] + EnergyMaster['potential_grid_charge']).cumsum().clip(0,storage_size_kW * storage_hours)

    #create customers load post storage
EnergyMaster['customer_load_post_storage'] = (EnergyMaster['Customer_Load_Post_PV'] - EnergyMaster['starting_storage_SOC_kWh']).clip_lower(0)

    #show discharged storage amount
EnergyMaster['storage_discharge'] = EnergyMaster['Customer_Load_Post_PV'] - EnergyMaster['customer_load_post_storage']

    #show charge storage amount
EnergyMaster['storage_charge'] = EnergyMaster['finished_storage_SOC_kWh'] - EnergyMaster['starting_storage_SOC_kWh']

    #show new customer load with storage charging
#EnergyMaster['final_customer_load'] = 
    

#Add solar LCOE
EnergyMaster['solar_lcoe'] = solar_lcoe

#Add diesel state of charge
#EnergyMaster['Diesel_SOC'] = EnergyMaster['CustomerLoad'].cumsum().clip(0,diesel_size_kW)

EnergyMaster.head(100)

# Create Randomized Grid Outages

In [ ]:
#variables
days_per_week_grid_down = 3
hoursdown = 4
percent_downtime = days_per_week_grid_down / (7.0*24/hoursdown)

#calculations
EnergyMaster['grid_down_probability'] = EnergyMaster['Customer_Load'].apply(lambda x: np.random.random())
def grid_is_down(rand_num):
    if rand_num < percent_downtime:
        return 1
    else:
        return 0
EnergyMaster['grid_down'] = EnergyMaster['grid_down_probability'].apply(grid_is_down)

# Calculate Combined Price of Diesel Scenario

Note: Storage-only scenario cost is the annual payment lease payment plus electricity price
Note2: Storage and PV cost is the annual lease payment plus the cost of solar plus electricity price

In [ ]:
#diesel scenario

EnergyMaster['cost_in_diesel'] = diesel_price
EnergyMaster['diesel_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','cost_in_diesel']]
EnergyMaster['diesel_scenario_price'].fillna(EnergyMaster['cost_in_diesel'],inplace=True)

EnergyMaster.head(100)

# Calculate Combined Price of Solar Scenario

In [ ]:
EnergyMaster['cost_in_solar'] = solar_lcoe
EnergyMaster['solar_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','solar_lcoe']]
EnergyMaster['solar_scenario_price'].fillna(EnergyMaster['solar_lcoe'],inplace=True)

# Calculate Totals

In [ ]:
#sum by year
EnergyMaster['timestamp_hour'] = pd.to_datetime(EnergyMaster['timestamp_hour'])
EnergyMaster['year'] = EnergyMaster['timestamp_hour'].dt.year

TotalSolarPrice = EnergyMaster.groupby(['year'])['solar_scenario_price'].sum()
TotalDieselPrice = (EnergyMaster.groupby(['year'])['diesel_scenario_price'].sum() + (diesel_capex / diesel_years))
TotalStoragePrice = (EnergyMaster.groupby(['year'])['solar_scenario_price'].sum() + storage_annual_payment)

print()
TotalStoragePrice

# Calculate Throughput 

In [ ]:
#storage change in charge

#fix variables with new storage categories above

#EnergyMaster['storage_change_charge'] = EnergyMaster['Storage_SOC'] - EnergyMaster['Storage_SOC_Post_Load']

#storage_throughput = EnergyMaster.groupby(['year'])['storage_change_charge'].sum()
#diesel_throughput = EnergyMaster.groupby(['year'])['grid_down'].sum()

print()
diesel_throughput

# Optimization

In [ ]:
from scipy import optimize

#variables we control:
pv_cpw = [1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4]
diesel_fuel_cost = range(1,4)

In [ ]:
# this creates a dataframe of all possible combinations of the variables above:

from itertools import product
costs_df = pd.DataFrame(list(product(pv_cpw,diesel_fuel_cost)), 
                     columns=['pv_cpw', 'diesel_fuel_cost'])
costs_df.head()diesel = []
storage = []

# now, for everyone of those combinations of cpw and fuel costs 
#we will calculate the energy cost for diesel and the energy cost for storage

for index, row in costs_df.iterrows():
    
    #these are the functions we ran above, but now doing them in a more structured fashion
    df = EnergyMaster.groupby(EnergyMaster.index / row['hoursdown']).sum() 
    df['grid_down_probability'] = df['kWh'].apply(lambda x: np.random.random())
    percent_downtime = row['daysdown'] / (7.0*24/row['hoursdown']) 
    df['grid_down'] = df['grid_down_probability'].apply(grid_is_down)
    kwh_grid_down = (df['grid_down']*df['kWh']).sum() #this gives us the total kWh that the grid is down

    
    #these calculate the diesel cost in this scenario for every row in the dataframe and appends to a list
    diesel_total_cost = (diesel_annual_payment + #lease costs
                     kwh_grid_down*diesel_om_kwh + #operations and maintenance
                     df['grid_down'].sum()*hoursdown*gallons_per_hour*row['costpergal']) #fuel
    diesel.append(diesel_total_cost)
        
    #now the storage costs   
    storage_total_cost = (storage_annual_payment + kwh_grid_down*storage_electricity_rate)
    storage.append(storage_total_cost)
    
costs_df['dieselcost'] = diesel
costs_df['storagecost'] = storage